# Testing Cleaning Data

In [ ]:
from bs4 import BeautifulSoup
import sys
import mwparserfromhell
import re
import pandas as pd
import copy
import time
import pandas as pd

In [2]:
sys.path.append('./ClonedModel/wmModel/wiki-detox/src/data_generation/diff_utils.py')
import diff_utils

### Cleaning Data
**Step 1** taking difference of texts  
**Step 2** using WikiMedia's `diff_utils` to clean texts


In [203]:
def diff_and_clean(data):
    ''' taking the diff and clean the text column'''
    
    titles = data.title.unique()
    for title in titles:
        data_subset = data[data.title == title]

        text_new = data_subset.text[1:]
        text_old = data_subset.text[:-1]
        text_diff = [data_subset.text.iloc[0]]

        for [new,old] in zip(text_new,text_old):
            text_diff.append(new.replace(old,''))
        # the diff_utils function requires a column named "insertion
        data.loc[data.title==title,'insertion'] = text_diff
    data = diff_utils.clean_and_filter(data)
    return data


def clean_and_diff(data):
    ''' 
    Inverse the order of taking diff and clean
    but experimentally it has no effect
    '''
    
    data['insertion'] = data['text']
    data = diff_utils.clean_and_filter(data)
    
    titles = data.title.unique()
    for title in titles:
        data_subset = data[data.title == title]

        text_new = data_subset.clean_diff[1:]
        text_old = data_subset.clean_diff[:-1]
        text_diff = [data_subset.clean_diff.iloc[0]]

        for [new,old] in zip(text_new,text_old):
            text_diff.append(new.replace(old,''))

        data.loc[data.title==title,'clean_diff'] = text_diff
    
    return data

## Demo

In [202]:
test_data = pd.read_csv('parsed.csv', sep='\t', nrows = 100)
test_data = diff_and_clean(test_data)

In [200]:
demo_idx = 53
original_text = test_data.loc[:,'text'][demo_idx]
diff_text = test_data.loc[:,'diff'][demo_idx]
clean_text = test_data.loc[:,'clean_diff'][demo_idx]
print('\t\t\toriginal\n\n %s\n\n\n'%original_text)
print('\t\t\tdiff\n\n %s\n\n\n'%diff_text)
print('\t\t\tcleaned\n\n %s'%clean_text)

			original

 == Wrong Name ==

The act is actually the Indians Arts and Crafts Act of 1990. There's a redirect page from the proper name to this page, but it seems more appropriate to move this page to reflect the name of the actual act. 

I removed the WikiProject India template, since that's obviously misplaced.
-[[User:Uyvsdi|Uyvsdi]] ([[User talk:Uyvsdi|talk]]) 00:57, 25 March 2009 (UTC)Uyvsdi

==Tipis included in the law?==
Does anybody know if [[tipi|tipis / teepees / tepes]] are part of this law?  I counted at least 58 tipi manufacturers in the United States and only about five(?) claimed to be Native. Dinkytown 23:18, 19 April 2009 (UTC)
::No. Anyone can make a tipi; just like anyone is welcome to paint a painting. The only time the law gets involved (it's totally unenforced/unenforceable anyway) is when an artist claims to be Native American but isn't enrolled or can't document their tribal heritage (with a CDIB card). -[[User:Uyvsdi|Uyvsdi]] ([[User talk:Uyvsdi|talk]]) 23:35